In [1]:
import numpy as np
import re
from functools import reduce
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold
import helpers
from nltk.corpus import wordnet, stopwords
import operator
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import RegexpTokenizer
# from math import log
# from options import WORD_FREQUENCIES

In [ ]:
with open('../twitter-datasets/train_pos.txt') as pos_in, open(
    '../twitter-datasets/train_neg.txt') as neg_in:
    pos_lines = pos_in.readlines()
    neg_lines = neg_in.readlines()
    pos_in.close()
    neg_in.close()
lines = pos_lines + neg_lines
lines = [line.replace('\n', '') for line in lines]
print(len(lines))
print(lines[0])
lines_copy = lines.copy()

In [ ]:
def count_occurence(text_list):
    
    def find_punct(text):
        return len(re.findall('! ', text))
    
    counter = 0
    for line in text_list:
        counter += find_punct(line)
    print(counter)
    
count_occurence(lines_copy)

In [1]:
def replace_emoji(text):
    rep_text = text
    rep_text = re.sub(':(-)?@', ' angry_emoji ', rep_text)
    rep_text = re.sub(':( )?\$', ' blushing_emoji ', rep_text)
    rep_text = re.sub('>:\)|>:D|>:-D|>;\)|>:-\)|}:-\)|}:\)|3:-\)|3:\)', ' devil_emoji ', rep_text) #done
    rep_text = re.sub('O:-\)|0:-3|0:3|0:-\)|0:\)|0;^\)', ' angel_emoji ', rep_text)
    rep_text = re.sub(':( )?\)+|\(+:|:-\)+|\(+-:|:\}| c : |:( )?O( )?\)|:( )?-( )?]|^( )?-( )?^', ' happy_emoji ', rep_text, flags=re.I)
    rep_text = re.sub(
        ':-D|:D|=D|=-D|8-D|8D|x-D|xD|X-D|=-D|:( )?d|:-d|>:d|=3|=-3|:\'-\)|:\'\)|\(\':|\[:|:\]', ' happy_emoji ', rep_text)    
    rep_text = re.sub('\)+:|:\(+|:-\(+|\)+-:|>:\[| : c |:\||:-\[', ' sad_emoji ', rep_text)
    rep_text = re.sub(':( )?\*|:( )?-( )?\*+|:x', ' kiss_emoji ', rep_text, flags=re.I) 
    rep_text = re.sub('<3', ' heart ', rep_text)
    rep_text = re.sub(';-\)|;\)|\*\)|\*-\)|;-\]|;]|;D|;\^\)', ' wink_emoji ', rep_text)
    rep_text = re.sub('>:P|:-P|:P|X-P|xp|=p|:b|:-b|;p| : p ', ' tongue_emoji ', rep_text, flags=re.I)
    rep_text = re.sub('>:O|:( )?-( )?O|:( )?O', ' surprise_emoji ', rep_text, flags=re.I) #done
    rep_text = re.sub(
    ':-\||<:-\||>( )?.( )?<|:( )?-( )?\/|:( )?-( )?\\\\|:S|:( )?\/|=\/|=( )?\\\\|:( )?\\\\|:( )?-( )?s|;( )?/|:( )?l ',
    ' skeptical_emoji ', rep_text) #done
    return rep_text

In [2]:
preproc.replace_emoji(':)')

' happy_emoji '

In [ ]:
'../twitter-datasets/words-by-frequency.txt'

In [ ]:

def emphasize_sentiment_words(tweet):
    """
    DESCRIPTION: 
                By using an opinion lexicon, if a tweet contained a positive or negative word
                 from that lexicon, it is emphasized respectively
    INPUT: 
            tweet: a tweet as a python string
    OUTPUT: 
            a transformed tweet as a python string
    """
    t = []
    for w in tweet.split():
        if w in positiveWords:
            t.append('positive ' + w)
        elif w in negativeWords:
            t.append('negative ' + w)
        else:
            t.append(w)
    return (" ".join(t)).strip()

### Cross validation

In [ ]:
count_vect = CountVectorizer() # stop_words = stopwords.words('english')
X = count_vect.fit_transform(lines)
vocab = count_vect.get_feature_names()

In [ ]:
y = np.zeros(shape=(len(lines)))
y[:len(pos_lines)] = 1
y[len(pos_lines):] = -1

In [ ]:
clf = LogisticRegression().fit(X, y)
kf = KFold(n_splits=4, shuffle=True, random_state=0)
scores = cross_val_score(clf, X, y, cv=kf)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

In [ ]:
# start_time = time.time()
# elapsed_time = divmod(round((time.time() - start_time)), 60)
# print('------\nElapsed time: {m} min {s} sec\n'.format(m=elapsed_time[0], s=elapsed_time[1]))